In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

<AxesSubplot:xlabel='count', ylabel='domain'>

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<AxesSubplot:xlabel='count', ylabel='posted_by'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

domain         \
                                            count unique   
domain               posted_by                             
americanthinker.com  Kaslin                   141      1   
                     MtnClimber               189      1   
                     SeekAndFind               54      1   
apnews.com           SeekAndFind                1      1   
                     ChicagoConservative27      1      1   
                     Oldeconomybuyer          109      1   
                     RandFan                    6      1   
                     blueplum                  14      1   
                     ebb tide                   1      1   
breitbart.com        SeekAndFind                2      1   
                     ChicagoConservative27    201      1   
                     RandFan                    1      1   
                     ebb tide                   2      1   
                     Red Badger                 3      1   
citizenfreepress.com MtnClimber                 3      1   
                     SeekAndFind               16      1   
                     Red Badger                36      1   
cnn.com              SeekAndFind                1      1   
                     ChicagoConservative27      1      1   
                     blueplum                  19      1   
                     Red Badger                 1      1   
                     nickcarraway               1      1   
dailymail.co.uk      SeekAndFind                4      1   
                     ChicagoConservative27      2      1   
                     RandFan                    8      1   
                     blueplum                  25      1   
                     Red Badger                 2      1   
foxnews.com          Kaslin                     1      1   
                     SeekAndFind                1      1   
                     ChicagoConservative27     15      1   
                     RandFan                    5      1   
                     blueplum                  30      1   
                     Red Badger                 2      1   
                     nickcarraway               1      1   
lifesitenews.com     ebb tide                  29      1   
                     Red Badger                 1      1   
msn.com              Kaslin                     1      1   
                     SeekAndFind                8      1   
                     Oldeconomybuyer            1      1   
                     RandFan                    1      1   
                     blueplum                  34      1   
                     Red Badger                 3      1   
                     nickcarraway              10      1   
news.yahoo.com       SeekAndFind                3      1   
                     ChicagoConservative27      2      1   
                     Oldeconomybuyer            3      1   
                     RandFan                    1      1   
                     Red Badger                 1      1   
                     nickcarraway               7      1   
newsbusters.org      ebb tide                   1      1   
nypost.com           SeekAndFind                3      1   
                     ChicagoConservative27    117      1   
                     RandFan                    4      1   
                     blueplum                  13      1   
                     ebb tide                   1      1   
                     Red Badger                 5      1   
                     nickcarraway              52      1   
nytimes.com          RandFan                    1      1   
                     blueplum                   1      1   
pjmedia.com          SeekAndFind               38      1   
                     Red Badger                 1      1   
redstate.com         Kaslin                     1      1   
                     SeekAndFind               67      1   
                     Red Badger                 1      1   
rumble.com           Ra

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

,timestamp,replies
title,,
Trump Urges Republicans to Sit Out Coming Elections,2021-10-14 00:16:36,170.0
MAGA supporters raise $2.4 million for jailed Marine Stuart Scheller — Then he insults President Trump and his family…,2021-10-04 02:34:06,154.0
What were your favorite Arcade games or Pinball from the 70s and early 80s ?,2021-10-18 04:04:04,153.0
